## Example: Working with raster data

In [ ]:
# import hydromt and initiate the test artifact data catalog
import hydromt

data_catalog = hydromt.DataCatalog()

Here, we illustrate some common GIS problems and how the functionality of the DataArray/Dataset [raster](../api.rst#raster-methods) accessor can be used. The data is accessed using the HydroMT [DataCatalog](../_generated/hydromt.data_catalog.DataCatalog.rst). For more information see the [Reading raster data](reading_raster_data.ipynb) example.

Note that this implementation is based on a very earlier version of [rioxarray](https://corteva.github.io/rioxarray/stable/) and in fact we are using some of rioxarray functionality and working towards replacing our duplicated functionality with that of rioxarray and will try to contribute new functionality to rioxarray. The original reason for the raster data accessor was that we needed some changes in the internals of the writing methods for PCRaster data which is not fully supported by its GDAL driver. Currently the key difference between both packages, besides the naming of the accessor and some methods, are in new methods that have been added over time by both packages and the way that the raster attribute data is stored. In HydroMT this attribute data is always stored in the `spatial_ref` coordinate of the DataArray/Dataset whereas rioxarray uses additional attributes of the RasterDataset class. 

### Geospatial attributes 

Using the [raster](../api.rst#raster-methods) accessor we can get (and set) many geospatial properties of regular raster datasets. 

In [ ]:
# get the GHS population raster dataset from the catalog
da = data_catalog.get_rasterdataset("ghs_pop_2015").rename("population")
da.raster.mask_nodata().reset_coords(drop=True).plot(vmax=200)

In [ ]:
# coordinate reference system
da.raster.crs

In [ ]:
# geospatial transform, see https://www.perrygeo.com/python-affine-transforms.html
da.raster.transform

In [ ]:
# names of x- and y dimensions
(da.raster.x_dim, da.raster.y_dim)

In [ ]:
# nodata value (or fillvalue) of a specific variable
da.raster.nodata

### Reproject (warp) raster data

A common preprocessing step to generate model data is to make sure all data is in the same CRS. The [.raster.reproject()](../_generated/hydromt.DataArray.raster.reproject.rst) are build on **rasterio** and **GDAL** and make this really easy for you. 

In this example we demonstrate how to reproject a population count grid. This grid should not be reprojected directly to other resolutions in order to conserve the total number of people. Therefore, we first derive the population density using [.raster.density_grid()](../_generated/hydromt.DataArray.raster.density_grid.rst) which can be reprojected and combined with the project grid cell area using [.raster.area_grid()](../_generated/hydromt.DataArray.raster.area_grid.rst) to calculate the final population count. Using this approach we only make a small error which we correct to preserve the total count. 

In [ ]:
from hydromt.gis_utils import utm_crs

utm = utm_crs(da.raster.bounds)
print(f"Destination CRS: {utm}")
da_pop = da.raster.mask_nodata()
da_pop_dens = da_pop.raster.density_grid().rename("population_dens")  # pop.m-2
da_pop_dens_utm = da_pop_dens.raster.reproject(
    method="bilinear", dst_crs=utm, dst_res=250
)
da_pop_utm = da_pop_dens_utm * da_pop_dens_utm.raster.area_grid()  # pop
bias = (da_pop.sum() / da_pop_utm.sum()).compute().item()
print(f"Error: {(1-bias)*100:.3f}%")
da_pop_utm_adj = da_pop_utm * bias  # bias correct
da_pop_utm_adj.name = "population"
da_pop_utm_adj.reset_coords(drop=True).plot(vmax=200)

### Zonal statistics

For many lumped models, zonal statistics are required to derive model parameters. The [.raster.zonal_stats()](../_generated/hydromt.DataArray.raster.zonal_stats.rst) method implements a range of statistics, but also allows for user defined statistics passed as a callable. Here we provide an example to get the population count per admin 3 level. HydroMT takes care that the vector data is reprojected to the raster data CRS if necessary.

In [ ]:
import xarray as xr
import numpy as np

gdf = data_catalog.get_geodataframe("gadm_level3", variables=["NAME_3"])
ds = xr.merge(
    [
        da_pop.raster.zonal_stats(gdf, stats=["sum"]) / 1e3,  # count [pop x1000]
        da_pop_dens.raster.zonal_stats(gdf, stats=["max", np.nanmax])
        * 1e6,  # density [pop km-2]
    ]
)
for var in ds.data_vars:
    gdf[var] = ds[var]

gdf.sort_values("population_sum", ascending=False).head()

In [ ]:
gdf.plot(
    "population_sum",
    scheme="NaturalBreaks",
    legend=True,
    legend_kwds=dict(fmt="{:.0f}", title="population [x1000]"),
    figsize=(6, 6),
)

### Interpolate nodata values

To create a continuos grid with values gaps in the data can be filled trough interpolation. HydroMT has the [.raster.interpolate_na()](../_generated/hydromt.DataArray.raster.interpolate_na.rst) method with several interpolation options available. For the nearest, linear and cubic interpolation the [scipy.interpolate.griddata()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.griddata.html) method is used. First, a mesh of all valid values surrounding the data gaps is build using voronoi triangulation. Then, values are calculated for each grid cell with a nodata value. Note that nearest method also extrapolates, while the other methods only interpolate gaps. A final method is based on [rasterio.fill.fillnodata()](https://rasterio.readthedocs.io/en/latest/api/rasterio.fill.html) method.

In [ ]:
da_soil = data_catalog.get_rasterdataset(
    "soilgrids", bbox=[12.7, 45.6, 13, 45.8], variables=["ph_sl1"]
)
da_soil.raster.mask_nodata().plot()

In [ ]:
# Note that data is only interpolated leaving a small nodata gap in the lower right corner. This can be extrapolated with the 'nearest' method.
da_soil.raster.interpolate_na(method="linear").raster.mask_nodata().plot()

### Reproject and merge

This example shows how to use the [.raster.reproject_like()](../_generated/hydromt.DataArray.raster.reproject_like.rst) method to align different datasets such that these are at identical grids and can be merged. 

In [ ]:
# step 1: read the data and mask nodata values
bbox = [12.2, 45.3, 13, 45.8]
da_dem = data_catalog.get_rasterdataset("merit_hydro", variables=["elevtn"], bbox=bbox)
da_dem = da_dem.raster.mask_nodata()
da_dem.attrs.update(units="m+EGM96")
print(f"resolution MERIT Hydro: {da_dem.raster.res[0]:.05f}")
da_bath = data_catalog.get_rasterdataset("gebco").raster.mask_nodata()
print(f"resolution GEBCO: {da_bath.raster.res[0]:.05f}")
da_mdt = data_catalog.get_rasterdataset("mdt_cnes_cls18").raster.mask_nodata()
print(f"resolution MDT: {da_mdt.raster.res[0]:.05f}")

plot_kwargs = dict(
    vmin=-50,
    vmax=50,
    cmap="coolwarm",
)
da_dem.plot(**plot_kwargs)

In [ ]:
# step 2: convert GEBCO to EGM96 ref and reproject to MERIT Hydro grid
da_bath_egm = da_bath + da_mdt.raster.reproject_like(da_bath)
da_bath_reproj = da_bath_egm.raster.reproject_like(da_dem, method="cubic")
print(f"resolution reprojected GEBCO: {da_bath_reproj.raster.res[0]:.05f}")
da_dem_merged = da_dem.where(da_dem.notnull(), da_bath_reproj)
da_dem_merged.plot(**plot_kwargs)

### Write raster to file

To write a dataset to a raster file the [.raster.to_raster()](../_generated/hydromt.DataArray.raster.to_raster.rst) can be used. By default the file is written in GeoTiff format. Each band is written as a layer of the raster file. A xarray.Dataset with multiple variables can be written to multiple files in a single folder, each with the name of the variable as basename with [.raster.to_mapstack()](../_generated/hydromt.Dataset.raster.to_mapstack.rst). 

Here, we use the merged DEM output of the previous example to write to file. To ensure the CRS and nodata metadata are written we first update these attributes of the data based on the original DEM data.

In [ ]:
da_dem_merged.raster.set_crs(da_dem.raster.crs)
da_dem_merged.raster.set_nodata(-9999)
da_dem_merged.raster.to_raster(
    "tmpdir/dem_merged.tif", tags={"history": "produced with HydroMT"}
)

In [ ]:
# use gdalinfo to check output. Note the CRS (GEOGCRS), Metadata and NoData Value
!gdalinfo tmpdir/dem_merged.tif